<a href="https://colab.research.google.com/github/FoxPowerGH/GenAI-Journey/blob/main/GenAI_gemini2_DOJO_WS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Configuración del Ambiente - Google CoLab**

## **Instalamos Google Gen AI SDK**

La documentación:  [Google AI for Developers](https://ai.google.dev/gemini-api/docs?hl=es-419)

In [ ]:
# Instalo las librerías requeridas en CoLab
%pip install -U -q 'google-genai'


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.7/130.7 kB 4.9 MB/s eta 0:00:00


In [ ]:
# Reiniciamos el kernel luego de la instalación para que el ambiente pueda acceder a los nuevos paquetes
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)


{'status': 'ok', 'restart': True}

## **Configuramos nuestra API Key**

In [ ]:
#Cargamos las dependencias requeridas
import os

try:
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# API KEY & Secrets
if COLAB:
    from google.colab import userdata
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

Note: using Google CoLab


## **Iniciamos el cliente del SDK**

In [ ]:
from google import genai
from google.genai import types

client = genai.Client(api_key=GOOGLE_API_KEY)


## **Escogemos el Modelo a utilizar**

In [ ]:
#Cargamos uno por defecto - Gemini 2.0 Flash model
MODEL_ID = "gemini-2.0-flash-exp"  # @param {type: "string"}

## **Comprobamos que podemos utilizar el LLM a partir del texto de un prompt simple**

In [ ]:
# Realizamos una prueba de conectividad
from IPython.display import Markdown
prompt = "¿Cuál es el planeta más grande en nuestro sistema solar?"

response = client.models.generate_content(
    model=MODEL_ID, contents=prompt
)

display(Markdown(response.text))

El planeta más grande de nuestro sistema solar es **Júpiter**.


# **Ejemplos prácticos**

## **Iniciar un chat de múltiples turnos**

In [ ]:
chat = client.chats.create(model=MODEL_ID)

In [ ]:
prompt = "Genera una pregunta de selección múltiple acerca de fundamentos de SOC L1"

response = chat.send_message(prompt)

display(Markdown(response.text))

¿Cuál de las siguientes tareas es **menos probable** que realice un Analista SOC L1 en su rol diario?

a) Monitorear alertas de seguridad y correlacionarlas.
b) Realizar análisis forense detallado de malware complejo.
c) Escalar incidentes a niveles superiores para su investigación.
d) Seguir playbooks y procedimientos operativos estándar (SOPs) predefinidos.


In [ ]:
response = chat.send_message("la respuesta es b")

display(Markdown(response.text))

Correcto. La respuesta es **b) Realizar análisis forense detallado de malware complejo.**

Aquí está la justificación:

*   **Analistas SOC L1 (Nivel 1)** generalmente son el primer punto de contacto en el SOC. Su enfoque principal está en la detección inicial, el triaje básico y la escalación. No suelen tener la experiencia ni la autorización para realizar análisis forenses profundos.

*   **a) Monitorear alertas de seguridad y correlacionarlas:** Esta es una tarea fundamental para identificar posibles incidentes.

*   **c) Escalar incidentes a niveles superiores para su investigación:**  Cuando un analista L1 identifica un incidente que requiere un análisis más profundo, lo escala a los analistas L2 o L3.

*   **d) Seguir playbooks y procedimientos operativos estándar (SOPs) predefinidos:** Los SOPs son cruciales para asegurar la consistencia y la eficiencia en las operaciones del SOC.

Por lo tanto, el análisis forense detallado de malware complejo es más propio de un Analista SOC L2 o L3, o incluso de un especialista en respuesta a incidentes.


## **Configurar parámetros del model**

In [ ]:
prompt = "Explícame los peligros del Internet, para mis hijos menores de edad."

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=types.GenerateContentConfig(
        temperature=0.4,            # Baja creatividad
        top_k=20,
        top_p= 0.5,
        candidate_count=1,
        seed=5,
        max_output_tokens=100,      # Tengo pocos token para responder
        stop_sequences=["STOP!"],
        presence_penalty=0.0,
        frequency_penalty=0.0,
    ),
)

display(Markdown(response.text))

¡Hola! Es muy importante hablar con tus hijos sobre los peligros de Internet, adaptando el lenguaje a su edad. Aquí te presento una explicación que puedes adaptar, dividida por temas y con ejemplos:

**1. Contacto con desconocidos (Peligro de grooming y ciberacoso):**

*   **Explicación:** "Imagina que en la calle no hablarías con cualquier persona, ¿verdad? Pues en Internet es igual. Hay personas que no son quienes

## **Establecer instrucciones del sistema**

In [ ]:
system_instruction = """
  Eres un asistente especializado en traducción.
  Tu misión es trauducir texto de español a inglés.
"""

prompt = "Me gustan las hamburguesas."

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=types.GenerateContentConfig(
        system_instruction=system_instruction,
    ),
)

display(Markdown(response.text))

I like hamburgers.


In [ ]:
prompt = "Favor traducir al frances el siguiente texto:   Me gustan los bagels."

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=types.GenerateContentConfig(
        system_instruction=system_instruction,
    ),
)

display(Markdown(response.text))

Por supuesto, aquí está la traducción de "Me gustan los bagels" al francés:

"J'aime les bagels."


In [ ]:
system_instruction = """
  Eres un asistente especializado en traducción.
  Tu misión es trauducir texto de español a inglés.
  Si te solicitan traducir a otro idioma, responde de forma educada y amigable tu misión.
"""

prompt = "Favor traducir al frances el siguiente texto:   Me gustan los bagels."

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=types.GenerateContentConfig(
        system_instruction=system_instruction,
    ),
)

display(Markdown(response.text))

Soy un asistente especializado en traducción de español a inglés. No puedo traducir a francés.


## **Filtros de seguridad**

Cuando haces una solicitud a Gemini, el contenido es analizado y se le asigna una calificación de seguridad. Podemos inspeccionar las calificaciones de seguridad del contenido generado imprimiendo las respuestas del modelo.

Los ajustes de seguridad están **DESACTIVADOS** por defecto, y los umbrales predeterminados de bloqueo están configurados en **BLOCK_NONE**.

Puedes utilizar safety_settings para ajustar la configuración de seguridad en cada solicitud que realices a la API.

Este ejemplo muestra cómo configurar el umbral de bloqueo en BLOCK_LOW_AND_ABOVE para todas las categorías:

In [ ]:

prompt = """
    Escribe una lista de 5 cosas odiosas, crueles y irrespetuosas que podría
    decirle al universo después de golpearme el dedo del pie en la oscuridad
"""

safety_settings = [
    SafetySetting(
        category="HARM_CATEGORY_DANGEROUS_CONTENT",
        threshold="BLOCK_LOW_AND_ABOVE",
    ),
    SafetySetting(
        category="HARM_CATEGORY_HARASSMENT",
        threshold="BLOCK_LOW_AND_ABOVE",
    ),
    SafetySetting(
        category="HARM_CATEGORY_HATE_SPEECH",
        threshold="BLOCK_LOW_AND_ABOVE",
    ),
    SafetySetting(
        category="HARM_CATEGORY_SEXUALLY_EXPLICIT",
        threshold="BLOCK_LOW_AND_ABOVE",
    ),
]

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=GenerateContentConfig(
        safety_settings=safety_settings,
    ),
)

# Response will be `None` if it is blocked.
print(response.text)
print(response.candidates[0].finish_reason)

for safety_rating in response.candidates[0].safety_ratings:
    print(safety_rating)


None
FinishReason.SAFETY
blocked=None category=<HarmCategory.HARM_CATEGORY_HATE_SPEECH: 'HARM_CATEGORY_HATE_SPEECH'> probability=<HarmProbability.NEGLIGIBLE: 'NEGLIGIBLE'> probability_score=3.966169e-05 severity=<HarmSeverity.HARM_SEVERITY_NEGLIGIBLE: 'HARM_SEVERITY_NEGLIGIBLE'> severity_score=None
blocked=None category=<HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: 'HARM_CATEGORY_DANGEROUS_CONTENT'> probability=<HarmProbability.NEGLIGIBLE: 'NEGLIGIBLE'> probability_score=7.5322026e-07 severity=<HarmSeverity.HARM_SEVERITY_NEGLIGIBLE: 'HARM_SEVERITY_NEGLIGIBLE'> severity_score=None
blocked=True category=<HarmCategory.HARM_CATEGORY_HARASSMENT: 'HARM_CATEGORY_HARASSMENT'> probability=<HarmProbability.MEDIUM: 'MEDIUM'> probability_score=0.2674951 severity=<HarmSeverity.HARM_SEVERITY_MEDIUM: 'HARM_SEVERITY_MEDIUM'> severity_score=0.2343337
blocked=None category=<HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: 'HARM_CATEGORY_SEXUALLY_EXPLICIT'> probability=<HarmProbability.NEGLIGIBLE: 'NEGLIGI